## Importing Libraries

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import time


device = torch.device('cuda')
device

device(type='cuda')

## Dataset & MODEL Modules

In [3]:
class StegoDataset(Dataset):
    def __init__(self, metadata_file, cover_dir, stego_dir, transform=None):
        self.metadata = pd.read_csv(metadata_file)
        self.cover_dir = cover_dir
        self.stego_dir = stego_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = self.metadata.iloc[idx]['image_name']
        is_stego = self.metadata.iloc[idx]['is_stego']
        
        # Load the image from appropriate directory
        if is_stego == 1:
            img_path = os.path.join(self.stego_dir, img_name)
        else:
            img_path = os.path.join(self.cover_dir, img_name)
        
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Label: 1 for stego, 0 for cover
        label = torch.tensor(is_stego, dtype=torch.float32)
        
        return image, label


def get_data_loaders(dataset_dir, batch_size=32, train_ratio=0.8, val_ratio=0.1):
    metadata_file = os.path.join(dataset_dir, 'metadata.txt')
    cover_dir = os.path.join(dataset_dir, 'cover')
    stego_dir = os.path.join(dataset_dir, 'stego')

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    dataset = StegoDataset(metadata_file, cover_dir, stego_dir, transform=transform)
    
    dataset_size = len(dataset)
    train_size = int(train_ratio * dataset_size)
    val_size = int(val_ratio * dataset_size)
    test_size = dataset_size - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    return train_loader, val_loader, test_loader

# YE NET

In [4]:
class SRM_conv2d(nn.Module):
    def __init__(self):
        super(SRM_conv2d, self).__init__()
        q = [4.0, 12.0, 2.0]
        filter1 = [[0, 0, 0, 0, 0],
                  [0, -1, 2, -1, 0],
                  [0, 2, -4, 2, 0],
                  [0, -1, 2, -1, 0],
                  [0, 0, 0, 0, 0]]
        filter2 = [[-1, 2, -2, 2, -1],
                  [2, -6, 8, -6, 2],
                  [-2, 8, -12, 8, -2],
                  [2, -6, 8, -6, 2],
                  [-1, 2, -2, 2, -1]]
        filter3 = [[0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 1, -2, 1, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]]
        
        srm_filters = [filter1, filter2, filter3]
        srm_weight = torch.zeros(3, 1, 5, 5)
        
        for idx, kernel in enumerate(srm_filters):
            srm_weight[idx, 0] = torch.FloatTensor(kernel)
        
        self.srm = nn.Conv2d(1, 3, kernel_size=5, stride=1, padding=2, bias=False)
        self.srm.weight = nn.Parameter(srm_weight, requires_grad=False)
    
    def forward(self, x):
        r, g, b = torch.split(x, 1, dim=1)
        
        r_srm = self.srm(r)
        g_srm = self.srm(g)
        b_srm = self.srm(b)
        
        # Concatenate results
        srm_output = torch.cat([r_srm, g_srm, b_srm], dim=1)
        return srm_output

class YeNet(nn.Module):
    def __init__(self):
        super(YeNet, self).__init__()
        
        # SRM preprocessing layer
        self.srm = SRM_conv2d()
        
        # First group of convolutional layers with stride=2 to reduce dimensions early
        self.conv1 = nn.Sequential(
            nn.Conv2d(9, 16, kernel_size=3, padding=1, stride=2), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Second group
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Third group
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Fourth group
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1), 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Fully connected layers with much fewer parameters
        self.fc = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        # SRM preprocessing
        out = self.srm(x)
        
        # Convolutional layers
        out = self.conv1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        
        # Global pooling
        out = self.global_pool(out)
        
        # Flatten
        out = out.view(out.size(0), -1)
        
        # Fully connected layers
        out = self.fc(out)
        
        return torch.sigmoid(out)  # Convert to probability

# SR Model

In [ ]:
class SRNet(nn.Module):
    """
    Lightweight implementation of SRNet architecture for steganalysis.
    Reduced parameters for easier GPU training.
    """
    def __init__(self):
        super(SRNet, self).__init__()
        
        # Reduced filters throughout the network
        
        # Preprocessing (reduced from 64 to 32 filters)
        self.preprocess = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        
        # Layer 1 (reduced from 64 to 32 filters)
        self.layer1 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        
        # Layer 2 (reduced from 128 to 64 filters)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        # Layer 3 (reduced from 256 to 96 filters)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 96, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=3, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU()
        )
        
        # Layer 4 (reduced from 512 to 128 filters)
        self.layer4 = nn.Sequential(
            nn.Conv2d(96, 128, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Fully connected layer (reduced from 512 to 128 filters)
        self.fc = nn.Linear(128, 1)
    
    def forward(self, x):
        out = self.preprocess(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return torch.sigmoid(out)

# Training

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=10, device='cuda',
          save_path='model.pth', accumulation_steps=1, use_amp=False, scaler=None):
    import time
    import torch
    from tqdm import tqdm
    import gc
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        optimizer.zero_grad()
        
        start_time = time.time()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        
        for batch_idx, (inputs, labels) in enumerate(train_pbar):
            inputs = inputs.to(device, non_blocking=True)  
            labels = labels.to(device, non_blocking=True)
            
            if use_amp:
                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels.view(-1, 1))
                    loss = loss / accumulation_steps  
            
                scaler.scale(loss).backward()
                
                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                outputs = model(inputs)
                loss = criterion(outputs, labels.view(-1, 1))
                loss = loss / accumulation_steps  
                loss.backward()
                
                if (batch_idx + 1) % accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()
            
            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()
            
            # Accumulate loss
            running_loss += loss.item() * accumulation_steps
            
    
            train_acc = 100 * correct / total
            train_pbar.set_postfix({
                'loss': f'{running_loss/(batch_idx+1):.3f}',
                'acc': f'{train_acc:.2f}%',
                'mem': f'{torch.cuda.memory_allocated()/1024**2:.1f}MB'
            })
            
    
            del inputs, labels, outputs, loss
            if batch_idx % 10 == 0:
                torch.cuda.empty_cache()
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Valid]')
        
        with torch.no_grad():  
            for inputs, labels in val_pbar:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                if use_amp:
                    with torch.amp.autocast(device_type='cuda'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels.view(-1, 1))
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels.view(-1, 1))
                
                val_loss += loss.item()
                
                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted.view(-1) == labels).sum().item()
                
                val_acc = 100 * val_correct / val_total
                val_pbar.set_postfix({
                    'loss': f'{val_loss/(val_pbar.n+1):.4f}',
                    'acc': f'{val_acc:.2f}%'
                })
                
                del inputs, labels, outputs, loss
            
            torch.cuda.empty_cache()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Acc: {100*correct/total:.2f}%, '
              f'Val Loss: {val_loss:.4f}, '
              f'Val Acc: {val_acc:.2f}%, '
              f'Time: {time.time()-start_time:.2f}s')
        
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f'Model saved to {save_path}')
    
    return model

# LSB 8 Train

In [11]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_8',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.01
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [ ]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net8.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Train]:   0%|          | 0/817 [00:00<?, ?it/s]

Epoch 1/10 [Valid]:   0%|          | 0/103 [00:00<?, ?it/s]/tmp/ipykernel_15365/1459737283.py:91: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.20it/s, loss=0.5726, acc=88.18%]


Epoch 1/10 - Train Loss: 0.6038, Train Acc: 81.02%, Val Loss: 0.5670, Val Acc: 88.18%, Time: 131.59s
Model saved to models/ye_net8.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.49it/s, loss=nan, acc=81.02%]  


Epoch 2/10 - Train Loss: 0.5507, Train Acc: 91.29%, Val Loss: nan, Val Acc: 81.02%, Time: 144.03s


...

Epoch 3/10 - Train Loss: 0.5405, Train Acc: 91.91%, Val Loss: 0.5168, Val Acc: 96.36%, Time: 137.30s
Model saved to models/ye_net8.pth


Epoch 4/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.30it/s, loss=0.5414, acc=93.36%]


Epoch 4/10 - Train Loss: nan, Train Acc: 92.23%, Val Loss: 0.5362, Val Acc: 93.36%, Time: 140.48s


Epoch 5/10 [Train]:   4%|▍         | 34/817 [00:06<02:19,  5.61it/s, loss=nan, acc=90.35%, mem=173.3MB]  


KeyboardInterrupt: 

# LSB 7 Train

In [ ]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_7',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [ ]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net7.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.06it/s, loss=0.4987, acc=99.27%]


Epoch 1/10 - Train Loss: 0.5189, Train Acc: 96.91%, Val Loss: 0.4987, Val Acc: 99.27%, Time: 141.81s
Model saved to models/ye_net7.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.07it/s, loss=0.4975, acc=99.79%]


Epoch 2/10 - Train Loss: 0.5057, Train Acc: 99.63%, Val Loss: 0.4975, Val Acc: 99.79%, Time: 152.23s
Model saved to models/ye_net7.pth


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.10it/s, loss=0.5028, acc=99.45%]


...

Epoch 4/10 - Train Loss: 0.5051, Train Acc: 99.74%, Val Loss: 0.4974, Val Acc: 99.72%, Time: 154.66s
Model saved to models/ye_net7.pth


Epoch 5/10 [Valid]: 100%|██████████| 103/103 [00:17<00:00,  5.94it/s, loss=0.4971, acc=99.69%]


Epoch 5/10 - Train Loss: 0.5049, Train Acc: 99.77%, Val Loss: 0.4971, Val Acc: 99.69%, Time: 149.68s
Model saved to models/ye_net7.pth


Epoch 6/10 [Train]:   4%|▍         | 32/817 [00:06<02:29,  5.25it/s, loss=0.497, acc=99.80%, mem=240.7MB]


KeyboardInterrupt: 

# LSB 6 Train

In [19]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_6',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [20]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net6.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.32it/s, loss=0.6999, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6936, Train Acc: 50.17%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 142.60s
Model saved to models/ye_net6.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.81it/s, loss=0.6999, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 140.16s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.17it/s, loss=0.6999, acc=48.18%]


Epoch 3/10 - Train Loss: 0.6932, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 128.92s


Epoch 4/10 [Train]:   5%|▍         | 40/817 [00:08<02:42,  4.77it/s, loss=0.693, acc=51.17%, mem=140.1MB]


KeyboardInterrupt: 

# LSB 6 SRNET

In [ ]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_6',
    batch_size=16,
    train_ratio=0.8,
    val_ratio=0.1
)

sr_net = SRNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.01
criterion = nn.BCEWithLogitsLoss()
optimizer_sr = torch.optim.Adam(sr_net.parameters(), lr=lr)

accumulation_steps = 4

In [ ]:
sr_net = train(
    model=sr_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_sr,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_sr, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/sr_net6.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 205/205 [00:31<00:00,  6.44it/s, loss=0.6931, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6935, Train Acc: 50.20%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 646.69s
Model saved to models/sr_net6.pth


Epoch 2/10 [Valid]: 100%|██████████| 205/205 [00:31<00:00,  6.41it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 654.69s


Epoch 3/10 [Train]:  92%|█████████▏| 1504/1634 [09:32<00:49,  2.63it/s, loss=0.693, acc=50.23%, mem=101.0MB]


KeyboardInterrupt: 

# LSB 5 Train

In [6]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_5',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [7]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net5.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.11it/s, loss=0.6930, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6933, Train Acc: 50.26%, Val Loss: 0.6930, Val Acc: 48.18%, Time: 170.21s
Model saved to models/ye_net5.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.11it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6932, Train Acc: 50.18%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 171.74s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.58it/s, loss=0.6931, acc=48.18%]


...

Epoch 8/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.83s


Epoch 9/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.53it/s, loss=0.6931, acc=48.18%]


Epoch 9/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.77s


Epoch 10/10 [Valid]: 100%|██████████| 103/103 [00:17<00:00,  5.94it/s, loss=0.6931, acc=48.18%]


Epoch 10/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.77s


# LSB 4 Train

In [8]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_4',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [9]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net4.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.59it/s, loss=0.6931, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6935, Train Acc: 50.16%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 170.51s
Model saved to models/ye_net4.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.65it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.16s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.62it/s, loss=0.6931, acc=48.18%]


...

Epoch 8/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 163.73s


Epoch 9/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.39it/s, loss=0.6931, acc=48.18%]


Epoch 9/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.45s


Epoch 10/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.39it/s, loss=0.6931, acc=48.18%]


Epoch 10/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.32s


# LSB 3 Train

In [10]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_3',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [11]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net3.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.45it/s, loss=0.6931, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6934, Train Acc: 50.20%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.91s
Model saved to models/ye_net3.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.14it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6932, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.34s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.61it/s, loss=0.6931, acc=48.18%]


...

Epoch 8/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.03s


Epoch 9/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.49it/s, loss=0.6931, acc=48.18%]


Epoch 9/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 163.37s


Epoch 10/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.43it/s, loss=0.6931, acc=48.18%]


Epoch 10/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 166.05s


# LSB 2 Train

In [13]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_2',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [14]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net2.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.46it/s, loss=0.6931, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6933, Train Acc: 50.22%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.77s
Model saved to models/ye_net2.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.53it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 163.64s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.65it/s, loss=0.6931, acc=48.18%]


...

Epoch 8/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.14s


Epoch 9/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.33it/s, loss=0.6931, acc=48.18%]


Epoch 9/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.42s


Epoch 10/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.38it/s, loss=0.6931, acc=48.18%]


Epoch 10/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 166.05s


# LSB 1 Train

In [15]:
train_loader, val_loader, test_loader = get_data_loaders(
    dataset_dir='dataset_1',
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1
)

ye_net = YeNet().to(device)

use_amp = hasattr(torch.cuda, 'amp')
scaler = torch.amp.GradScaler('cuda') if use_amp else None

lr = 0.001
criterion = nn.BCEWithLogitsLoss()
optimizer_ye = torch.optim.Adam(ye_net.parameters(), lr=lr)

accumulation_steps = 4 

In [16]:
ye_net = train(
    model=ye_net,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_ye,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ye, 'min', patience=3, factor=0.5),
    num_epochs=10,
    device=device,
    save_path='models/ye_net1.pth',
    accumulation_steps=accumulation_steps,
    use_amp=use_amp,
    scaler=scaler
)

Epoch 1/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.45it/s, loss=0.6931, acc=48.18%]


Epoch 1/10 - Train Loss: 0.6933, Train Acc: 50.18%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.11s
Model saved to models/ye_net1.pth


Epoch 2/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.44it/s, loss=0.6931, acc=48.18%]


Epoch 2/10 - Train Loss: 0.6932, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.17s


Epoch 3/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.41it/s, loss=0.6931, acc=48.18%]


...

Epoch 8/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.22s


Epoch 9/10 [Valid]: 100%|██████████| 103/103 [00:16<00:00,  6.32it/s, loss=0.6931, acc=48.18%]


Epoch 9/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 164.67s


Epoch 10/10 [Valid]: 100%|██████████| 103/103 [00:15<00:00,  6.54it/s, loss=0.6931, acc=48.18%]

Epoch 10/10 - Train Loss: 0.6931, Train Acc: 50.19%, Val Loss: 0.6931, Val Acc: 48.18%, Time: 165.38s


# Testing

In [7]:
def classify_image(image_path, model, device):
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        prediction = (output > 0.5).float()
    
    return prediction.item()

In [15]:
cover = './dataset_8/cover/18157.png'
stego = './dataset_8/stego/18157.png'
prediction_cover = classify_image(cover, ye_net, device)
prediction_stego = classify_image(stego, ye_net, device)
print(f'Prediction for cover: {"Stego" if prediction_cover == 1 else "Cover"}')
print(f'Prediction for stego: {"Stego" if prediction_stego == 1 else "Cover"}')

Prediction for cover: Cover
Prediction for stego: Stego
